<a href="https://colab.research.google.com/github/gksdydcks/Ai/blob/main/output.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import json
import csv

In [ ]:
# 특정 폴더의 모든 파일 가져오기
def get_all_files(directory):
    all_files = []
    for root, _, files in os.walk(directory):  # 하위 디렉토리 포함 탐색
        for file in files:
            all_files.append(os.path.join(root, file))  # 전체 경로 추가
    return all_files

In [ ]:
# 외관 디렉토리 경로
# 여기에 실제 디렉토리 경로 입력
fender_ed_source_directory = r"D:\data\data\source_data\fender\Exterior_damage"
fender_source_ed_files = get_all_files(fender_ed_source_directory)
# 여기에 실제 디렉토리 경로 입력
fender_ed_label_directory = r"D:\data\data\label_data\fender\Exterior_damage"
fender_source_ed_labels = get_all_files(fender_ed_label_directory)

# 결과 CSV 파일 경로
fender_ed_data = r"D:/data/data/output/fender/fender_ed_data.csv"

# 함수 실행
data_info_to_csv(fender_source_ed_labels,
                 fender_source_ed_files, fender_ed_data)

파일수정

In [ ]:
def data_info_to_csv(directory1, directory2, output_csv):
    global train_json_dataset
    train_json_dataset = []

    # JSON 파일에서 'quality'를 숫자로 변환하는 매핑
    quality_to_label = {"불량품": 0, "양품": 1}

    # 이미지 파일 이름과 경로 매핑
    image_name_to_path = {os.path.basename(path): path for path in directory2}

    for json_file in directory1:
        with open(json_file, 'r', encoding='utf-8-sig') as f:
            data = json.load(f)

        # JSON 파일에서 images 섹션 처리
        image_id_to_name = {img["id"]: img["file_name"] for img in data.get("images", [])}

        for annotation in data.get("annotations", []):
            # annotation의 image_id로 file_name 가져오기
            image_id = annotation.get("image_id")
            file_name = image_id_to_name.get(image_id)

            # 이미지 파일 경로 찾기
            path = image_name_to_path.get(file_name, "Unknown")

            # quality, part, bbox 처리
            quality = annotation['attributes'].get('quality')
            part = annotation['attributes'].get('part')
            bbox = annotation.get('bbox', [])
            bbox_str = str(bbox) if bbox else "[]"

            # 데이터 추가
            train_json_dataset.append([
                path,
                quality_to_label.get(quality, -1),
                part,
                bbox_str
            ])

    # CSV로 저장
    with open(output_csv, mode='w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        # 헤더 작성
        writer.writerow(["path", 'quality', "part", "bboxes"])
        # 데이터 작성
        writer.writerows(train_json_dataset)
